# [Module 4.1] Deploy Built-in TFS Image


여기서는 다음과 같은 작업을 합니다.

- 모델 아티펙트 (model.tar.gz) 파일을 S3에서 로컬에 다운로드
- TF Saved_Model 의 정의를 확인
- SageMaker Model 생성
- Endpoint 생성
- Inference의 Request Serializer and Deserializer 생성
- 프리딕터 생성
- 셈플 데이타로 추론

---
이 노트북은 약 10분 정도 소요 됩니다.


필요한 프로그램 설치

In [1]:
!pip install -q --upgrade pip
!pip install -q wrapt --upgrade --ignore-installed
!pip install -q tensorflow==2.1.0
!pip install -q transformers==2.8.0
!pip install -q sagemaker==1.56.1

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

astroid 2.3.3 requires wrapt==1.11.*, but you'll have wrapt 1.12.1 which is incompatible.


In [2]:
import boto3
import sagemaker
import pandas as pd
import tensorflow as tf
import os

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [3]:
%store -r training_job_name

In [4]:
print(training_job_name)

tensorflow-training-2020-08-17-09-58-09-919


## 모델 정의 확인

아래의 세개의 셀은 모델을 로컬에 다운로드 받고 텐서플로우의 Saved Model의 정의 (입력, 출력)를 확인하는 코드 입니다. 확인을 원하시면 아래 주석을 제거하고 실행하시면 됩니다.

In [5]:
# model_download = 'model'
# os.makedirs(model_download, exist_ok=True)

In [6]:
# !aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz {model_download}/model.tar.gz

In [7]:
# !tar -xvzf   {model_download}/model.tar.gz
# !saved_model_cli show --all --dir ./tensorflow/saved_model/0/

## SageMaker Model 생성

In [8]:
import os
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
              role=role,
              framework_version='2.0.0') # Elastic Inference does not yet support TF 2.1.0 as of sagemaker==1.56.1

## Endpoint 생성

In [9]:
instance_type='ml.m4.xlarge'

deployed_model = model.deploy(initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=True)



-------------!

In [10]:
endpoint_name = deployed_model.endpoint
print('Endpoint name:  {}'.format(endpoint_name))

Endpoint name:  tensorflow-inference-2020-08-17-10-16-41-783


## Inference Request Serializer and Deserializer 생성

In [11]:
class RequestHandler(object):
    import json
    
    def __init__(self, tokenizer, max_seq_length):
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __call__(self, instances):
        transformed_instances = []

        for instance in instances:
            encode_plus_tokens = tokenizer.encode_plus(instance,
                                                       pad_to_max_length=True,
                                                       max_length=self.max_seq_length)

            input_ids = encode_plus_tokens['input_ids']
            input_mask = encode_plus_tokens['attention_mask']
            segment_ids = [0] * self.max_seq_length

            transformed_instance = {"input_ids": input_ids, 
                                    "input_mask": input_mask, 
                                    "segment_ids": segment_ids}

            transformed_instances.append(transformed_instance)

        transformed_data = {"instances": transformed_instances}

        return json.dumps(transformed_data)
    
class ResponseHandler(object):
    import json
    import tensorflow as tf
    
    def __init__(self, classes):
        self.classes = classes
    
    def __call__(self, response, accept_header):
        import tensorflow as tf

        response_body = response.read().decode('utf-8')

        response_json = json.loads(response_body)

        log_probabilities = response_json["predictions"]

#        predicted_classes = []

        # Convert log_probabilities => softmax (all probabilities add up to 1) => argmax (final prediction)
#         for log_probability in log_probabilities:
#             softmax = tf.nn.softmax(log_probability)    
#             predicted_class_idx = tf.argmax(softmax, axis=-1, output_type=tf.int32)
#             predicted_class = self.classes[predicted_class_idx]
#             predicted_classes.append(predicted_class)

        return log_probabilities    

## Predictor 생성

In [12]:
import json
from sagemaker.tensorflow.serving import Predictor
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

request_handler = RequestHandler(tokenizer=tokenizer,
                                 max_seq_length=32)

response_handler = ResponseHandler(classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

predictor = Predictor(endpoint_name=endpoint_name,
                      sagemaker_session=sess,
                      serializer=request_handler,
                      deserializer=response_handler,
                      content_type='application/json',
                      model_name='saved_model',
                      model_version=0)

## Inference 실행

In [13]:
from TweetUtil import TweetUtil

tweet_util = TweetUtil()
tweet_util.load_emoji_data('emoji_to_idx.pickle')
emoji = tweet_util.get_emo_class_label(3)
print(emoji)

emoji_to_idx is loaded
😂


In [14]:
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
test_file_path = 'data/test/tweet_file_test.csv'
test_df = pd.read_csv(test_file_path)
sample_df = test_df.sample(10)
sample_df

,TWEET,LABEL
2323,yesterday the air conditioner was on in the bu...,8
5954,loving these colors,5
7638,thank you,0
2743,vapemail thanks dude can't wait to open them ...,4
9033,niggas really forget who was down when nobody...,7
432,okay i followed you so try again please,4
4330,bless up,5
7147,that deep sleepy voice,3
3885,i've noticed that social media is the cause to...,9
150,i always turn it off here i love the s6 finale,5


In [15]:
def show_top_N_label(score_list, topN):

    import numpy as np

    top_n_idx = np.argsort(score_list)[-topN:]
    top_n_values = [score_list[i] for i in top_n_idx]
    
    top_n_idx_list = top_n_idx.tolist()
    top_n_idx_list.reverse()
    top_n_values = [score_list[i] for i in top_n_idx_list]    
    
    return top_n_idx_list


In [20]:
import tensorflow as tf
import json

columns = ['TWEET', 'LABEL']
topN = 5
for tweet, label in zip(sample_df.TWEET.values, sample_df.LABEL.values):
    # print("label: {}, tweet: {}".format(label, tweet))
    
    reviews = [tweet]
    
#     print("reviews: \n", reviews)



    predicted_classes = predictor.predict(reviews)[0]
#    predicted_classes = predictor.predict(reviews)    
    predicted_classes = show_top_N_label(predicted_classes, topN)

    print('tweet: {} \nGround_truth- {}:{}\n '.format(
        tweet,
        label, 
        tweet_util.get_emo_class_label(label))
         )    
    

    print('Prediction: {},{},{},{},{},{},{},{},{},{} \n '.format(
        predicted_classes[0], 
        tweet_util.get_emo_class_label(predicted_classes[0]),
        predicted_classes[1], 
        tweet_util.get_emo_class_label(predicted_classes[1]),
        predicted_classes[2], 
        tweet_util.get_emo_class_label(predicted_classes[2]),                                       
        predicted_classes[3], 
        tweet_util.get_emo_class_label(predicted_classes[3]),                                      
        predicted_classes[4], 
        tweet_util.get_emo_class_label(predicted_classes[4]),                                      
        
        ))    
        



tweet: as if flynn said jill you don't know what it's like being 12 can you even remember being 12 mate i'm 18 not 80 
Ground_truth- 8:🙄
 
Prediction: 3,😂,7,😭,9,🤔,8,🙄,4,😊 
 
tweet: so 80 hr to cuddle with strangers welp might have to work my way into the cuddling industry  
Ground_truth- 9:🤔
 
Prediction: 8,🙄,7,😭,3,😂,9,🤔,4,😊 
 
tweet:  car freestyle session 
Ground_truth- 2:🔥
 
Prediction: 6,😩,2,🔥,4,😊,8,🙄,1,💕 
 
tweet:  how can people be jealous of a innocent lil baby like wtf especially if she he not even here yet just sad son  
Ground_truth- 9:🤔
 
Prediction: 8,🙄,3,😂,4,😊,5,😍,0,❤ 
 
tweet: got punched last night just because i was standing in the the wrong place at the wrong time 
Ground_truth- 3:😂
 
Prediction: 8,🙄,9,🤔,6,😩,2,🔥,7,😭 
 
tweet: finally off 
Ground_truth- 8:🙄
 
Prediction: 3,😂,8,🙄,4,😊,7,😭,5,😍 
 
tweet:  all the time 
Ground_truth- 7:😭
 
Prediction: 4,😊,8,🙄,2,🔥,7,😭,0,❤ 
 
tweet:  i don't know if y'all ready 
Ground_truth- 2:🔥
 
Prediction: 3,😂,9,🤔,5,😍,8,🙄,4,😊 
 
tweet:  sa